In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from batcher import Container
import cifar10
%matplotlib inline

In [2]:
ims, labels = cifar10.get_train_data()
images = Container(ims, labels)

In [3]:
test_x, test_y = cifar10.get_test_data()

In [4]:
sess = tf.InteractiveSession()

In [5]:
channels = 3

In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [7]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_3x3(x):
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

In [8]:
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
x_p = tf.placeholder(tf.float32)
x_drop = tf.nn.dropout(x, x_p)

y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [9]:
conv1_filters = 96

W_conv1 = weight_variable([5, 5, channels, conv1_filters])
b_conv1 = bias_variable([conv1_filters])

h_conv1 = tf.nn.relu(conv2d(x_drop, W_conv1) + b_conv1)
h_pool1 = max_pool_3x3(h_conv1)

pool1_p = tf.placeholder(tf.float32)
h_pool1_drop = tf.nn.dropout(h_pool1, pool1_p)

In [10]:
conv2_filters = 128

W_conv2 = weight_variable([5, 5, conv1_filters, conv2_filters])
b_conv2 = bias_variable([conv2_filters])

h_conv2 = tf.nn.relu(conv2d(h_pool1_drop, W_conv2) + b_conv2)
h_pool2 = max_pool_3x3(h_conv2)

pool2_p = tf.placeholder(tf.float32)
h_pool2_drop = tf.nn.dropout(h_pool2, pool2_p)

In [11]:
conv3_filters = 256

W_conv3 = weight_variable([5, 5, conv2_filters, conv3_filters])
b_conv3 = bias_variable([conv3_filters])

h_conv3 = tf.nn.relu(conv2d(h_pool2_drop, W_conv3) + b_conv3)
h_pool3 = max_pool_3x3(h_conv3)

pool3_p = tf.placeholder(tf.float32)
h_pool3_drop = tf.nn.dropout(h_pool3, pool3_p)

In [12]:
fc_units = 2048

In [13]:
W_fc1 = weight_variable([4 * 4 * conv3_filters, fc_units])
b_fc1 = bias_variable([fc_units])

h_pool3_flat = tf.reshape(h_pool3_drop, [-1, 4 * 4 * conv3_filters])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

fc1_p = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, fc1_p)

In [14]:
W_fc2 = weight_variable([fc_units, fc_units])
b_fc2 = bias_variable([fc_units])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

fc2_p = tf.placeholder(tf.float32)
h_fc2_drop = tf.nn.dropout(h_fc2, fc2_p)

In [15]:
W_logit = weight_variable([fc_units, 10])
b_logit = bias_variable([10])

y_conv = tf.matmul(h_fc2_drop, W_logit) + b_logit

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

for i in range(20000):
    batch_x, batch_y = images.next_batch(500)
    if i%100 == 0:
        train_accuracy = accuracy.eval(
            feed_dict={
                x: batch_x, 
                y_: batch_y,
                x_p: 1.,
                pool1_p: 1.,
                pool2_p: 1.,
                pool3_p: 1.,
                fc1_p: 1.,
                fc2_p: 1.
            }
        )
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(
        feed_dict={
            x: batch_x, 
            y_: batch_y,
            x_p: 0.9,
            pool1_p: 0.75,
            pool2_p: 0.75,
            pool3_p: 0.5,
            fc1_p: 0.5,
            fc2_p: 0.5
        }
    )

step 0, training accuracy 0.11
step 100, training accuracy 0.174
step 200, training accuracy 0.15
step 300, training accuracy 0.178
step 400, training accuracy 0.174
step 500, training accuracy 0.132
step 600, training accuracy 0.104
step 700, training accuracy 0.09


In [ ]:
chunk_size = 500

chunk_accuracies = [
    accuracy.eval(
        feed_dict={
            x: test_x[start:start+chunk_size], 
            y_: test_y[start:start+chunk_size],
            x_p: 1.,
            pool1_p: 1.,
            pool2_p: 1.,
            pool3_p: 1.,
            fc1_p: 1.,
            fc2_p: 1.
        }
    )
    for start in range(0,len(test_y),chunk_size)
]

acc = np.mean(chunk_accuracies)

print("test accuracy %g"%acc)